# <span style="color:green">Fixed</span>: Can not use a method of an instance if the instance and the type are defined separately
- [bug](https://github.com/yunabe/lgo/issues/11)

In [ ]:
type Data struct {
    value string
}

func (d Data) Value() string {
    return d.value
}

type WithValue interface {
    Value() string
}

In [ ]:
d := Data{"hello"}

In [3]:
// Got a link error because of https://github.com/golang/go/issues/22998
d.Value()

hello


In [ ]:
// This works without any problem.
WithValue(d).Value()

hello


# <span style="color:green">Fixed</span>: goroutines should be canceled when the main routine ends
- [bug](https://github.com/yunabe/lgo/issues/6)

In [ ]:
import (
    "fmt"
)

i := 0
go func() {
    defer func() {
        fmt.Println("Canceled")
    }()
    for i = 0;; i++ {}
}()

Canceled


# <span style="color:green">Fixed</span>: The entire process terminated when a go routine panics
- [bug](https://github.com/yunabe/lgo/issues/5)

In [ ]:
import (
    "fmt"
    "time"
)

go func() {
    panic("die!")
}()

time.Sleep(100 * time.Millisecond)
fmt.Println("main done")

panic: die!

goroutine 26 [running]:
runtime/debug.Stack(0xc4203f9eb8, 0x7fc12a067bc0, 0xc42003a010)
	/usr/lib/go-1.8/src/runtime/debug/stack.go:24 +0x7b
github.com/yunabe/lgo/core.FinalizeGoRoutine()
	/home/yunabe/local/gocode/src/github.com/yunabe/lgo/core/core.go:81 +0x64
panic(0x7fc12a067bc0, 0xc42003a010)
	/usr/lib/go-1.8/src/runtime/panic.go:489 +0x2e7
github.com/yunabe/lgo/sess7b2274696d65223a313530383530393332323132383131303736397d/exec8.lgo_init.func1.1()
	/home/yunabe/local/gocode/src/github.com/yunabe/lgo/sess7b2274696d65223a313530383530393332323132383131303736397d/exec8/src.go:13 +0x66
github.com/yunabe/lgo/sess7b2274696d65223a313530383530393332323132383131303736397d/exec8.lgo_init.func1()
	/home/yunabe/local/gocode/src/github.com/yunabe/lgo/sess7b2274696d65223a313530383530393332323132383131303736397d/exec8/src.go:14 +0x40
created by github.com/yunabe/lgo/sess7b2274696d65223a313530383530393332323132383131303736397d/exec8.lgo_init
	/home/yunabe/local/gocode/src/github.com/yu

main done
10
<nil>


# <span style="color:green">Fixed</span>: interface is not working
If you invoke a method through an interface in lgo, it crashes with `runtime error: invalid memory address or nil pointer dereference` ([bug](https://github.com/yunabe/lgo/issues/3)).

In [1]:
import (
    "fmt"
)

type Hello interface {
    SayHello()
}

type person struct {
    name string
}

func (p *person) SayHello() {
    fmt.Printf("Hello, I'm %s.\n", p.name)
}

p := person{"yunabe"}
fmt.Println("---- 1 ----")
p.SayHello()

var h Hello = &p
fmt.Println("---- 2 ----")
h.SayHello()

---- 1 ----
Hello, I'm yunabe.
---- 2 ----
Hello, I'm yunabe.


# <span style="color:green">Fixed</span>: GC crashes with a fatal error
The following crashes with `fatal error: found bad pointer in Go heap (incorrect use of unsafe or cgo?)` message ([bug](https://github.com/yunabe/lgo/issues/2)).

In [1]:
import (
    "fmt"
    "log"
    "runtime"
    "runtime/debug"
)

type MyData struct {
    b []byte
}

func (m *MyData) Size() int {
    return len(m.b)
}

func NewMyData() *MyData {
    return &MyData{
        b: make([]byte, 10 * (1 << 20)),
    }
}

var l []*MyData
for i := 0; i < 100; i++ {
    d := NewMyData()
    l = append(l, d)
}
l = nil
debug.FreeOSMemory()
runtime.GC()